In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-mutational-learning/LY16_test_data.csv
/kaggle/input/ml-mutational-learning/media-4.xlsx
/kaggle/input/ml-mutational-learning/LY16_train_data.csv
/kaggle/input/ml-mutational-learning/ACE2_test_data.csv
/kaggle/input/ml-mutational-learning/media-1.xlsx
/kaggle/input/ml-mutational-learning/REGN87_test_data.csv
/kaggle/input/ml-mutational-learning/LY16_unseen.tsv
/kaggle/input/ml-mutational-learning/P0DTC2.fasta.txt
/kaggle/input/ml-mutational-learning/mmc2.csv
/kaggle/input/ml-mutational-learning/REGN87_unseen.tsv
/kaggle/input/ml-mutational-learning/ACE2_train_data.csv
/kaggle/input/ml-mutational-learning/REGN87_train_data.csv
/kaggle/input/ml-mutational-learning/encoded_seqs_train.npy
/kaggle/input/ml-mutational-learning/REGN33_train_data.csv
/kaggle/input/ml-mutational-learning/encoded_seqs_unseen.npy
/kaggle/input/ml-mutational-learning/REGN33_test_data.csv
/kaggle/input/ml-mutational-learning/media-3.xlsx
/kaggle/input/ml-mutational-learning/encoded_seqs_test.npy
/k

In [2]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import random
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [3]:

SEED = 0xDEAD
random.seed(SEED)
np.random.seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)

In [4]:
train = pd.read_csv("/kaggle/input/ml-mutational-learning/REGN33_train_data.csv")
test = pd.read_csv("/kaggle/input/ml-mutational-learning/REGN33_test_data.csv")
unseen = pd.read_csv("/kaggle/input/ml-mutational-learning/REGN33_unseen.tsv", sep = '\t')

In [9]:
train.head()

,Unnamed: 0,junction_aa,v_call,consensus_count,j_call,clonal_frequency,Label,Distance,Antibody
0,157131,GNTGFNCYCPIKNYGFQRTGGLGW,IGHV_RBD,1,IGHJ_RBD,0.000008,0,10,REGN33
1,158034,KNTGFNCYLPLVRYGFQTTAGGDW,IGHV_RBD,1,IGHJ_RBD,0.000008,0,9,REGN33
2,52097,KNTGFNCYRPLNTYGFWRTSGINY,IGHV_RBD,1,IGHJ_RBD,0.000007,1,9,REGN33
3,122003,KNKGFNCYDPITRYGFWPTNGSNW,IGHV_RBD,1,IGHJ_RBD,0.000008,0,9,REGN33
4,110735,KNRGFNCYGPLVAYGFQRTCGMGW,IGHV_RBD,1,IGHJ_RBD,0.000008,0,8,REGN33


In [10]:
#list(train['junction_aa'])

In [11]:
train.head()

,Unnamed: 0,junction_aa,v_call,consensus_count,j_call,clonal_frequency,Label,Distance,Antibody
0,157131,GNTGFNCYCPIKNYGFQRTGGLGW,IGHV_RBD,1,IGHJ_RBD,0.000008,0,10,REGN33
1,158034,KNTGFNCYLPLVRYGFQTTAGGDW,IGHV_RBD,1,IGHJ_RBD,0.000008,0,9,REGN33
2,52097,KNTGFNCYRPLNTYGFWRTSGINY,IGHV_RBD,1,IGHJ_RBD,0.000007,1,9,REGN33
3,122003,KNKGFNCYDPITRYGFWPTNGSNW,IGHV_RBD,1,IGHJ_RBD,0.000008,0,9,REGN33
4,110735,KNRGFNCYGPLVAYGFQRTCGMGW,IGHV_RBD,1,IGHJ_RBD,0.000008,0,8,REGN33


In [12]:
encoded_seqs_train = np.load('/kaggle/input/ml-mutational-learning/encoded_seqs_train.npy')

In [13]:
encoded_seqs_test = np.load('/kaggle/input/ml-mutational-learning/encoded_seqs_test.npy')

In [14]:
encoded_seqs_unseen = np.load('/kaggle/input/ml-mutational-learning/encoded_seqs_unseen.npy')

In [56]:
#

In [34]:
def get_low_high(df, arr):
    arr = np.moveaxis(arr, 1, -1)
    high_var_X, high_var_y = [], []
    low_var_X, low_var_y = [], []
    labels = list(test['Label'])
    for i, (encode, dist) in enumerate(zip(arr, list(df['Distance']))):
        if dist >= 5:
            high_var_X.append(encode)
            high_var_y.append(labels[i])
        else:
            low_var_X.append(encode)
            low_var_y.append(labels[i])
    high_var_X = torch.from_numpy(np.array(high_var_X))
    low_var_X = torch.from_numpy(np.array(low_var_X))
    high_var_y = torch.from_numpy(np.array(high_var_y))
    low_var_y = torch.from_numpy(np.array(low_var_y))
    return high_var_X, low_var_X, high_var_y, low_var_y
    

In [35]:
X_test_hi, X_test_lw, y_test_hi, y_test_lw = get_low_high(test, encoded_seqs_test)

In [17]:
def get_data(df, arr):
    labels = torch.from_numpy(np.array(df['Label']))
    arr = np.moveaxis(arr, 1, -1)
    features = torch.from_numpy(arr)
    return features, labels
    

In [18]:
X_train, y_train = get_data(train, encoded_seqs_train)
X_test, y_test = get_data(test, encoded_seqs_test)
X_unseen, y_unseen = get_data(unseen, encoded_seqs_unseen)

In [19]:
class My_Data(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        sample = {'features': self.features[idx], 'labels': self.labels[idx]}
        return sample

In [20]:
train_dataset = My_Data(X_train, y_train)
test_dataset = My_Data(X_test, y_test)
unseen_dataset = My_Data(X_unseen, y_unseen)

In [36]:
test_lw = My_Data(X_test_lw, y_test_lw)
test_hi = My_Data(X_test_hi, y_test_hi)

In [37]:
trainloader = DataLoader(train_dataset, batch_size=128,
                        shuffle=True, num_workers=2)
testloader = DataLoader(test_dataset, batch_size=128,
                        shuffle=False, num_workers=2)
unseenloader = DataLoader(unseen_dataset, batch_size=128,
                        shuffle=False, num_workers=2)

In [38]:
testloader_lw = DataLoader(test_lw, batch_size=128,
                        shuffle=False, num_workers=2)
testloader_hw = DataLoader(test_hi, batch_size=128,
                        shuffle=False, num_workers=2)

In [24]:
ex = next(iter(trainloader))['features']

In [25]:
ex.shape

torch.Size([128, 40, 223])

In [17]:
class CNN_1(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels = 40, out_channels = 32, kernel_size=3),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels = 32, out_channels = 128, kernel_size=7),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels = 128, out_channels = 256, kernel_size=7), 
            nn.LeakyReLU(),
            nn.Conv1d(in_channels = 256, out_channels = 1024, kernel_size=13), 
            nn.LeakyReLU(),
            nn.Conv1d(in_channels = 1024, out_channels = 128, kernel_size=17),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size=17), 
            nn.LeakyReLU(), 
            nn.Flatten(),
            nn.Dropout(0.2)
        )
        self.cl = nn.Sequential(
            nn.Linear(21120, 1)
        )

    def forward(self, x):
        x = self.cnn(x)
        prediction = self.cl(x)
        return prediction

In [26]:
from tqdm.notebook import tqdm, trange

def training(model, criterion, optimizer, num_epochs, trainloader, testloader):
    for e in trange(num_epochs, leave=False):
        model.train()
        num_iter = 0
        train_loss = 0
        for batch in trainloader:
            optimizer.zero_grad()
            input_embeds = batch["features"].to(device)
            labels = batch["labels"].to(device)
            labels = labels.unsqueeze(1)
            labels = labels.float()
            prediction = model(input_embeds)
            loss = criterion(prediction, labels)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
            num_iter += 1
        print(f"Train_Loss: {train_loss / num_iter}")
        valid_loss = 0
        num_iter = 0
        model.eval()
        with torch.no_grad():
            for batch in testloader:
                input_embeds = batch["features"].to(device)
                labels = batch["labels"].to(device)
                labels = labels.unsqueeze(1)
                labels = labels.float()
                prediction = model(input_embeds)
                loss = criterion(prediction, labels)
                valid_loss += loss.item()
                num_iter += 1
        print(f"Valid Loss: {valid_loss / num_iter}")

In [22]:
class CNN_2(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels = 40, out_channels = 32, kernel_size=3),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels = 32, out_channels = 128, kernel_size=7),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels = 128, out_channels = 256, kernel_size=7), 
            nn.LeakyReLU(),
            nn.Conv1d(in_channels = 256, out_channels = 1024, kernel_size=13), 
            nn.LeakyReLU(),
            nn.Conv1d(in_channels = 1024, out_channels = 128, kernel_size=17),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size=17), 
            nn.LeakyReLU(), 
            nn.Flatten()
        )
        self.cl = nn.Sequential(
            nn.Linear(21120, 1)
        )

    def forward(self, x):
        x = self.cnn(x)
        prediction = self.cl(x)
        return prediction

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNN_2().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 12
training(model, criterion, optimizer, num_epochs, trainloader, testloader)

  0%|          | 0/12 [00:00<?, ?it/s]

Train_Loss: 0.681648334559608
Valid Loss: 0.3399031407776333
Train_Loss: 0.26706953139856776
Valid Loss: 0.29777125006630306
Train_Loss: 1129.8221935065264
Valid Loss: 4.10284324600583
Train_Loss: 11.785809234101722
Valid Loss: 0.9882792969544728
Train_Loss: 1.2184571047095543
Valid Loss: 0.582750735112599
Train_Loss: 247.0202799783425
Valid Loss: 261.34563525971913
Train_Loss: 9.807482229204888
Valid Loss: 5.285503380639213
Train_Loss: 1.4582733938668637
Valid Loss: 1.020587006069365
Train_Loss: 5.325572439806258
Valid Loss: 3.8732391856965567
Train_Loss: 0.8229195058345795
Valid Loss: 3.304591630754017
Train_Loss: 279.35633472969874
Valid Loss: 445.9953206380208
Train_Loss: 42.93445087747371
Valid Loss: 36.98541388738723


In [24]:
def evaluate_model(loader):
    m = nn.Sigmoid()
    model.eval()
    ground_truth = []
    all_prob = []
    all_predict = []
    with torch.no_grad():
        for batch in loader:
            input_embeds = batch["features"].to(device)
            labels = batch["labels"].to(device)
            prediction = model(input_embeds)
            prediction = prediction.squeeze(1)
            pred = torch.where(prediction > 0.5, 1, 0)
            all_predict.append(pred.cpu().numpy())
            prob = m(prediction)
            all_prob.append(prob.cpu().numpy())
            ground_truth.append(labels.cpu().numpy())
    all_prob = np.concatenate(all_prob)
    ground_truth = np.concatenate(ground_truth)
    all_predict = np.concatenate(all_predict)
    return all_prob, all_predict , ground_truth

In [40]:
def print_metrics(**kwargs):
    return pd.DataFrame(kwargs, index = [0])

In [100]:
all_prob, all_predict, ground_truth = evaluate_model(testloader)
recall = recall_score(ground_truth, all_predict)
acc = accuracy_score(ground_truth, all_predict)
prec = precision_score(ground_truth, all_predict)
auc = roc_auc_score(ground_truth, all_prob)
print_metrics(recall = recall, accracy = acc, precision = prec, roc_auc = auc)

,recall,accracy,precision,roc_auc
0,0.890887,0.914358,0.934591,0.974635


In [27]:
class CNNModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=1):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(embed_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1),
            nn.Flatten(),
        )
        self.cl = nn.Sequential(
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        x = self.cnn(x)
        prediction = self.cl(x)
        return prediction

In [30]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNNModel(embed_size = 40, hidden_size = 80).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 14

In [31]:
training(model, criterion, optimizer, num_epochs, trainloader, testloader)

  0%|          | 0/14 [00:00<?, ?it/s]

Train_Loss: 0.27371431601808427
Valid Loss: 0.22531269625538872
Train_Loss: 0.23070336884640633
Valid Loss: 0.2356003402244477
Train_Loss: 0.22185298592803326
Valid Loss: 0.20951084522973923
Train_Loss: 0.21693346239626407
Valid Loss: 0.20638860237030757
Train_Loss: 0.21182404661432225
Valid Loss: 0.2044152543658302
Train_Loss: 0.20814336169748865
Valid Loss: 0.2088742611663682
Train_Loss: 0.20551726252157637
Valid Loss: 0.20028955049458005
Train_Loss: 0.2020006177193941
Valid Loss: 0.19972550769646963
Train_Loss: 0.1992841557381635
Valid Loss: 0.20774741861082258
Train_Loss: 0.19772238511037318
Valid Loss: 0.1971179417201451
Train_Loss: 0.19511707580470025
Valid Loss: 0.20802403958070845
Train_Loss: 0.1932183241669802
Valid Loss: 0.19893965628885088
Train_Loss: 0.19052900044207877
Valid Loss: 0.19386171875965028
Train_Loss: 0.18756860607957587
Valid Loss: 0.20865409104597002


In [32]:
def evaluate_model(loader):
    m = nn.Sigmoid()
    model.eval()
    ground_truth = []
    all_prob = []
    all_predict = []
    with torch.no_grad():
        for batch in loader:
            input_embeds = batch["features"].to(device)
            labels = batch["labels"].to(device)
            prediction = model(input_embeds)
            prediction = prediction.squeeze(1)
            pred = torch.where(prediction > 0.5, 1, 0)
            all_predict.append(pred.cpu().numpy())
            prob = m(prediction)
            all_prob.append(prob.cpu().numpy())
            ground_truth.append(labels.cpu().numpy())
    all_prob = np.concatenate(all_prob)
    ground_truth = np.concatenate(ground_truth)
    all_predict = np.concatenate(all_predict)
    return all_prob, all_predict , ground_truth

In [41]:
all_prob, all_predict, ground_truth = evaluate_model(testloader_lw)
recall = recall_score(ground_truth, all_predict)
acc = accuracy_score(ground_truth, all_predict)
prec = precision_score(ground_truth, all_predict)
auc = roc_auc_score(ground_truth, all_prob)
print_metrics(recall = recall, accracy = acc, precision = prec, roc_auc = auc)

,recall,accracy,precision,roc_auc
0,0.972973,0.938931,0.923077,0.984175


In [42]:
all_prob, all_predict, ground_truth = evaluate_model(testloader_hw)
recall = recall_score(ground_truth, all_predict)
acc = accuracy_score(ground_truth, all_predict)
prec = precision_score(ground_truth, all_predict)
auc = roc_auc_score(ground_truth, all_prob)
print_metrics(recall = recall, accracy = acc, precision = prec, roc_auc = auc)

,recall,accracy,precision,roc_auc
0,0.948345,0.922266,0.900844,0.974232
